In [23]:
import kfp
from kfp import compiler
import kfp.components as comp
import kfp.dsl as dsl

In [24]:
EXPERIMENT_NAME = 'lightweight-component-training'

## Define a Pipeline

In [25]:
def concat(first: str, second: str) -> str:
   return first + second

In [26]:
concat_operation = comp.func_to_container_op(concat)

In [27]:
def reverse(text: str) -> str:
    return text[::-1]

In [28]:
reverse_operation = comp.func_to_container_op(reverse)

In [29]:
@dsl.pipeline(
   name='Text transformation pipeline',
   description='A pipeline that performs several string operations.'
)

def text_transformation_pipeline(first,second):
    concat_task = concat_operation(first, second)  
    reverse_task = reverse_operation(concat_task.output)

## Compile Pipeline

In [30]:
pipeline_func = text_transformation_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'

import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

## Run Pipeline

In [31]:
arguments = {'first': 'dess', 'second': 'erts'}

import kfp
client = kfp.Client()
#experiment = client.create_experiment(EXPERIMENT_NAME)
experiment = client.get_experiment(experiment_name=EXPERIMENT_NAME)

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)
